# NENA to TF

This notebook will be used to develop code for converting texts from .nena format to Text-Fabric. The parser has principally been written by Hannes Vlaardingerbroek. Many thanks to him for his hard work on it. Updates and refinements have been added by Cody Kingham.

In [1]:
import os
import sys
import collections
import re
import unicodedata
from pathlib import Path
from tf.convert.walker import CV
from tf.fabric import Fabric

# path to parser
parserpath = f'../../nena_corpus/parse_nena/'
sys.path.append(parserpath)
from nena_parser import NenaLexer, NenaParser

# paths
VERSION = '0.01'
OUT_DIR = Path(f'../tf/{VERSION}')
data_dir = Path(f'../../nena_corpus/nena/{VERSION}')
dialect_dirs = list(Path(data_dir).glob('*'))

In [2]:
TF = Fabric(locations=[str(OUT_DIR)], silent=True)
cv = CV(TF)

## Parse NENA

The NENA Parser delivers the text as structured morphemes, which can then be processed into a TF graph. We do that below by opening each source text, retrieving its parsed form, and begin each iteration. 

In [3]:
lexer = NenaLexer()
parser = NenaParser()

In [4]:
# dialect2file2parsed = collections.defaultdict(lambda: collections.defaultdict())

# nparsed = 0

# for dialect in sorted(dialect_dirs):    
#     print()
#     print(dialect.name)
#     for file in sorted(dialect.glob('*.nena')):
#         with open(file, 'r') as infile:
#             text = infile.read()
#             print(f'parsing: {file.name}')
#             parse = parser.parse(lexer.tokenize(text))
#             nparsed += 1
#             dialect2file2parsed[dialect.name][file.name] = parse
            

# print('\n', nparsed, 'texts ready for conversion')

In [5]:
# linenum, elements = dialect2file2parsed['Urmi_C']['Village Life.nena'][1][0][
# eg_morph = elements[0]

In [6]:
# dialect2file2parsed['Barwar']['A Hundred Gold Coins.nena'][1][0][0]

# Metadata

In [7]:
slotType = 'letter'

otext = {
    'sectionTypes': 'dialect,text,line',
    'sectionFeatures': 'dialect,title,number',
    'fmt:text-orig-full': '{text}{end}',
    'fmt:text-trans-full': '{trans_f}{etrans_f}',
    'fmt:text-trans-lite': '{trans_l}{etrans_l}',
}

description = ''.join("""
The NENA linguistic corpus is derived from decades of 
field work by Prof. Geoffrey Khan and his students.
""".split('\n'))

generic = {
    'origin': 'Cambridge University, Faculty of Asian and Middle Eastern Studies',
    'author': 'Geoffrey Khan et al.',
    'editors': 'Cody Kingham, James Strachan, Hannes Vlaardingerbroek',
    'researchers': 'Paul Noorlander, Dorota Molin',
    'source': description,
    'url': 'https://github.com/CambridgeSemiticsLab/nena_tf',
}

intFeatures = {'number'}

d = 'about'

featureMeta = {
    'dialect': {d: 'name of a dialect in Northeastern Neo-Aramaic'},
    'title': {d: 'title of a text (story)'},
    'version': {d: 'version of the story if there are multiple instances of the same story'},
    'number': {d: 'sequential number of a paragraph or line within a text or paragraph, respectively'},
    'text': {d: 'plain text representation of a letter, morpheme, or word'},
    'end': {d: 'space, punctuation, or other stylistic text at the end of a morpheme or word'},
    'trans_f': {d: 'full, one-to-one transcription of a letter, morpheme, or word'},
    'trans_l': {d: 'lite transcription of a letter, morpheme, or word, without vowel accents'},
    'etrans_f': {d: 'full transcription of punctuation or other stylistic text at the end of a morpheme or word; see also trans_f'},
    'etrans_l': {d: 'lite transcription of punctuation or other stylistic text at the end of a morpheme or word, excluding intonation boundary markers; see also trans_l'},
    'speaker': {d: 'name or initials of person speaking a morpheme or word; see also informant'},
    'footnotes': {d: 'explanatory footnote on a morpheme or text'},
    'lang': {d: 'language of a morpheme foreign to a text'},
    'foreign': {d: 'indicates whether a morpheme is foreign to a text; see also lang'},
    'comment': {d: 'explanatory comment inserted in the text, stored on a morpheme'},
    'continued_from': {d: 'text is a follow-up to the named text'},
    'informant': {d: 'name of person who spoke these words'},
    'place': {d: 'place a text was recorded'},
    'source': {d: 'name of the file from which a text was converted'},
    'text_id': {d: 'id of a text within its original publication; can overlap between publications'},
}

In [8]:
# Two tables are presented below
# they are largely identical, but edits
# are made as necessary on a char by char basis.
# The tables are used by the function `trans` to 
# convert the full text

trans_full = {
    # non-latin vowels
    '\u0131': '1',  # 0x0131 ı dotless i
    '\u0251': '@',  # 0x0251 ɑ alpha
    '\u0259': '3',  # 0x0259 ə schwa
    '\u025B': '$',  # 0x025B ɛ open e
    # vowel accents
    '\u0300': '`',  # 0x0300 à grave
    '\u0301': "'",  # 0x0301 á acute
    '\u0304': '_',  # 0x0304 ā macron
    '\u0306': '%',  # 0x0306 ă breve
    '\u0308': '"',  # 0x0308 ä diaeresis
    '\u0303': '~',  # 0x0303 ã tilde
    '\u02C8': '', # 0x2c8 ˈ small vertical line
    # non-latin consonants
    '\u00F0': '6',  # 0x00F0 ð eth
    '\u025F': '&',  # 0x025F ɟ small dotless j with stroke
    '\u0248': '!',  # 0x0248 Ɉ capital J with stroke
    '\u03B8': '8',  # 0x03B8 θ greek theta
    '\u02B8': '7',  # 0x02B8 ʸ small superscript y
    '\u02BE': '}',  # 0x02BE ʾ right half ring (alaph)
    '\u02BF': '{',  # 0x02BF ʿ left half ring (ayin)
    # consonant diacritics
    '\u207A': '+',  # 0x207A ⁺ superscript plus
    '\u030C': '<',  # 0x030C x̌ caron
    '\u0302': '^',  # 0x0302 x̂ circumflex
    '\u0307': ';',  # 0x0307 ẋ dot above
    '\u0323': '.',  # 0x0323 x̣ dot below
    '\u032D': '>',  # 0x032D x̭ circumflex below
    
    # punctuation
    '\u02C8': '|', # 0x2c8 ˈ small vertical line
}

trans_lite = {
    # non-latin vowels
    '\u0131': '1',  # 0x0131 ı dotless i
    '\u0251': 'a',  # 0x0251 ɑ alpha
    '\u0259': '3',  # 0x0259 ə schwa
    '\u025B': 'e',  # 0x025B ɛ open e
    # vowel accents
    '\u0300': '',  # 0x0300 à grave
    '\u0301': '',  # 0x0301 á acute
    '\u0304': '',  # 0x0304 ā macron
    '\u0306': '',  # 0x0306 ă breve
    '\u0308': '',  # 0x0308 ä diaeresis
    '\u0303': '',  # 0x0303 ã tilde
    '\u02C8': '', # 0x2c8 ˈ small vertical line
    # non-latin consonants
    '\u00F0': '6',  # 0x00F0 ð eth
    '\u025F': '&',  # 0x025F ɟ small dotless j with stroke
    '\u0248': '!',  # 0x0248 Ɉ capital J with stroke
    '\u03B8': '8',  # 0x03B8 θ greek theta
    '\u02B8': '7',  # 0x02B8 ʸ small superscript y
    '\u02BE': '}',  # 0x02BE ʾ right half ring (alaph)
    '\u02BF': '{',  # 0x02BF ʿ left half ring (ayin)
    # consonant diacritics
    '\u207A': '+',  # 0x207A ⁺ superscript plus
    '\u030C': '',  # 0x030C x̌ caron
    '\u0302': '',  # 0x0302 x̂ circumflex
    '\u0307': '',  # 0x0307 ẋ dot above
    '\u0323': '',  # 0x0323 x̣ dot below
    '\u032D': '',  # 0x032D x̭ circumflex below
    # others
    '\u02c8': '',  # 0x02c8 ˈ vertical line
}

def trans(s, table, mark_punct=True):
    '''
    Transcribes a text.
    '''
    s = unicodedata.normalize('NFD', s)
    # mark punctuation 
    if mark_punct:
        s = re.sub('([\n,.!?:;/])', r'/\g<1>', s, 1)
    return ''.join([table.get(c, c) for c in s])

# Converter

Build a TF Walker class that can walk over the NENA parsed data and fit the text graph.

In [9]:
[1][:-1]

[]

In [10]:
def make_footnotes(fn_dict):
    """Format footnote dict into string"""
    if fn_dict:
        return '; '.join(
            f'[^{num}]: {txt}' for num, txt in fn_dict.items()
        )
    else:
        return None

def make_wordfeats(mfeat_list, ignore={}):
    """Convert a list of morpheme feature dicts into one for a word.
    
    Features stored on a word must be inherited in special ways
    for words. For example, a word's "end" feature should be 
    the last morpheme, not all of the ends. Those features 
    are specially processed here.
    """
    
    word_fs = collections.defaultdict(set)

    # add features
    for mfeats in mfeat_list:
        for feat,val in mfeats.items():
            if feat in ignore:
                continue
            else:
                word_fs[feat].add(val)

    # handle special cases
    word_fs['end'] = mfeat_list[-1]['end']
    word_fs['etrans_f'] = mfeat_list[-1]['etrans_f']
    word_fs['etrans_l'] = mfeat_list[-1]['etrans_l']
    word_fs['text'] = ''.join(
        mf['text']+mf['end'] for mf in mfeat_list
    )
    # add transcription with end, leaving off the end from the 
    # last morpheme
    trans_parts = [('trans_f', 'etrans_f'), ('trans_l', 'etrans_l')]
    for trans, end in trans_parts:
        word_fs[trans] = ''
        for i,mf in enumerate(mfeat_list):
            word_fs[trans] += mf[trans]
            if i+1 != len(mfeat_list):
                word_fs[trans] += mf[end]
    
    # convert to strings and handle duplicates
    for feat,val in word_fs.items():
        if type(val) == set:
            val = {v for v in val if v}
            if val:
                word_fs[feat] = ' '.join(val)
            else:
                word_fs[feat] = None
        
    return word_fs
    
def director(CV):
    """Walk the source data and produce a TF graph"""
    
    info = TF.tm.info
    
    for dialect_dir in sorted(dialect_dirs):  
        
        # make dialect node
        dialect = cv.node('dialect')
        dia = dialect_dir.name
        cv.feature(dialect, dialect=dia)
        
        for file in sorted(dialect_dir.glob('*.nena')):
            
            info(f'processing: [{file}]')
            
            with open(file, 'r') as infile:
                nena_text = infile.read()
            
            # parse the .nena format
            header, paragraphs = parser.parse(lexer.tokenize(nena_text))
            
            # -- begin TF node creation --
            
            # cv.node initializes a node object
            # all slots added in between its creation and 
            # termination will be considered embedded within
            # this node; same is true of following cv.node calls
            text = cv.node('text')
            cv.feature(text, **header) # adds features to supplied node
            title = header['title']
            
            for i, para in enumerate(paragraphs):
                
                # TODO: Process footnotes here
                if len(para[0]) != 2:
                    continue
                
                # make paragraph node
                paragraph = cv.node('paragraph')
                cv.feature(paragraph, number=i+1)
                
                for line_num, line_elements in para:
                    
                    # make line nodes
                    line = cv.node('line')
                    cv.feature(line, number=line_num)
                    
                    # Make linguistic nodes by parsing morphemes.
                    # This must be done iteratively and composed
                    # based on characters at the end of each morpheme. 
                    # Punctuation signals intonation/subsentence/sentence 
                    # boundaries; spaces and hyphens signal word bounds. 
                    # This is handled in the loop below.
                    word = cv.node('word')
                    inton = cv.node('inton')
                    subsentence = cv.node('subsentence')
                    sentence = cv.node('sentence')
                    word_features = [] # store morphs feats here for processing

                    for i, elem in enumerate(line_elements):
                        
                        is_end = i+1 == len(line_elements)

                        # add morphemes as slots
                        # 'slot' being the most basic element
                        if elem.__class__.__name__ == 'Morpheme':
                            
                            # make morpheme node
                            morph = cv.node('morpheme')
                            
                            # access/prepare morph features
                            fs = elem.__dict__
                            trailer = elem.trailer.replace('/', '\n')
                            
                            # package & edit morph features for cv
                            # NB: None values are ignored by default
                            feats = {
                                'text': ''.join(elem.value),
                                'trans_f': trans(''.join(elem.value), trans_full),
                                'trans_l': trans(''.join(elem.value), trans_lite),
                                #'letters': ' '.join(elem.value),
                                'end': trailer,
                                'etrans_f': trans(trailer, trans_full),
                                'etrans_l': trans(trailer, trans_lite),
                                'speaker': fs.get('speaker') or header.get('informant'),
                                'footnotes': make_footnotes(fs.get('footnotes', {})),
                                'lang': fs.get('lang'),
                                'foreign': str(fs.get('foreign')) if fs.get('foreign') else None,
                            }
                            
                            # make letter slots
                            # creation of a slot simultaneously 
                            # embeds it within all active nodes
                            for i, let in enumerate(elem.value):
                                # letter features
                                letfs = {
                                    'text': let,
                                    'trans_f': trans(let, trans_full),
                                    'trans_l': trans(let, trans_lite),
                                    'end': '',
                                    'etrans_f': '',
                                    'etrans_l': '',                                        
                                }
                                if i+1 == len(elem.value):
                                    letfs['end'] = trailer
                                    letfs['etrans_f'] = feats['etrans_f']
                                    letfs['etrans_l'] = feats['etrans_l']
                                letter = cv.slot()
                                cv.feature(letter, **letfs)
                                cv.terminate(letter)
                            
                            word_features.append(feats)
                            cv.feature(morph, **feats)
                            cv.terminate(morph)
                                
                            # -- trigger linguistic node endings --
                            
                            # word ending
                            if (not re.match('^$|^[-=]$', trailer)) or is_end:
                                cv.feature(word, **make_wordfeats(word_features))
                                word_features = []
                                cv.terminate(word)
                                if not is_end:
                                    word = cv.node('word')

                            # intonation group ending
                            if re.search('\u02c8', trailer):
                                cv.terminate(inton)
                                if not is_end:
                                    inton = cv.node('inton')
                            
                            # subsentence ending
                            if re.search('[,;:\u2014\u2013]', trailer):
                                cv.terminate(subsentence)
                                if not is_end:
                                    subsentence = cv.node('subsentence')
                            
                            # sentence ending
                            elif re.search('[.!?]', trailer):
                                cv.terminate(subsentence)
                                cv.terminate(sentence)
                                if not is_end:
                                    subsentence = cv.node('subsentence')
                                    sentence = cv.node('sentence')
                                
                        # add other elements
                        else:
                            kind, data = elem
                            if kind == 'footnote':
                                cv.feature(text, footnote=make_footnotes(data))
                            else:
                                cv.feature(morph, **{kind:str(data)})
                    
                    # sanity check for un-closed words, itons, subsentences, sentences
                    # due either to lack of proper punctuation in the source text (to be fixed later)
                    # or due to non-morpheme elements intervening in the iteration
                    unclosed = {'inton','sentence', 'subsentence', 'word'} & cv.activeTypes()
                    if unclosed:
                        sys.stderr.write(f'force-closing types {unclosed} in {title} ln {line_num}\n')
                        cv.terminate(word)
                        cv.terminate(inton)
                        cv.terminate(subsentence)
                        cv.terminate(sentence)
                        
                    # -- trigger section node endings --
                    cv.terminate(line)
                cv.terminate(paragraph)
            cv.terminate(text)
        cv.terminate(dialect)

## Test good


In [11]:
good = cv.walk(
    director,
    slotType,
    otext=otext,
    generic=generic,
    intFeatures=intFeatures,
    featureMeta=featureMeta,
    warn=True,
    force=False,
)

  0.00s Importing data from walking through the source ...
   |     0.00s Preparing metadata... 
   |     0.00s No structure nodes will be set up
   |   SECTION   TYPES:    dialect, text, line
   |   SECTION   FEATURES: dialect, title, number
   |   STRUCTURE TYPES:    
   |   STRUCTURE FEATURES: 
   |   TEXT      FEATURES:
   |      |   text-orig-full       end, text
   |      |   text-trans-full      etrans_f, trans_f
   |      |   text-trans-lite      etrans_l, trans_l
   |     0.01s OK
   |     0.00s Following director... 
   |     0.00s processing: [../../nena_corpus/nena/0.01/Barwar/A Hundred Gold Coins.nena]
   |     0.11s processing: [../../nena_corpus/nena/0.01/Barwar/A Man Called Čuxo.nena]
   |     0.28s processing: [../../nena_corpus/nena/0.01/Barwar/A Tale of Two Kings.nena]
   |     0.45s processing: [../../nena_corpus/nena/0.01/Barwar/A Tale of a Prince and a Princess.nena]
   |     0.94s processing: [../../nena_corpus/nena/0.01/Barwar/Baby Leliθa.nena]


force-closing types {'sentence', 'subsentence'} in A Tale of a Prince and a Princess ln 32


   |     1.21s processing: [../../nena_corpus/nena/0.01/Barwar/Dəmdəma.nena]
   |     1.37s processing: [../../nena_corpus/nena/0.01/Barwar/Gozali and Nozali.nena]


force-closing types {'sentence', 'subsentence'} in Gozali and Nozali ln 1


   |     2.24s processing: [../../nena_corpus/nena/0.01/Barwar/I Am Worth the Same as a Blind Wolf.nena]
   |     2.36s processing: [../../nena_corpus/nena/0.01/Barwar/Man Is Treacherous.nena]
   |     2.48s processing: [../../nena_corpus/nena/0.01/Barwar/Measure for Measure.nena]
   |     2.51s processing: [../../nena_corpus/nena/0.01/Barwar/Nanno and Jəndo.nena]


...

   |       23s processing: [../../nena_corpus/nena/0.01/Urmi_C/Village Life.nena]


force-closing types {'inton'} in Village Life (6) ln 34


   |       23s processing: [../../nena_corpus/nena/0.01/Urmi_C/Vineyards.nena]
   |       23s processing: [../../nena_corpus/nena/0.01/Urmi_C/Weddings and Festivals.nena]


force-closing types {'inton'} in Village Life ln 1
force-closing types {'inton'} in Village Life ln 5
force-closing types {'inton'} in Village Life ln 18
force-closing types {'inton'} in Village Life ln 20


   |       23s processing: [../../nena_corpus/nena/0.01/Urmi_C/Weddings.nena]
   |       24s processing: [../../nena_corpus/nena/0.01/Urmi_C/When Shall I Die?.nena]
   |       24s processing: [../../nena_corpus/nena/0.01/Urmi_C/Women Are Stronger Than Men.nena]
   |       24s processing: [../../nena_corpus/nena/0.01/Urmi_C/Women Do Things Best.nena]
   |       25s "edge" actions: 0
   |       25s "feature" actions: 756315
   |       25s "node" actions: 294155
   |       25s "resume" actions: 0
   |       25s "slot" actions: 539381
   |       25s "terminate" actions: 833720
   |          2 x "dialect" node 
   |      35985 x "inton" node 
   |     539381 x "letter" node  = slot type
   |       2544 x "line" node 
   |     120148 x "morpheme" node 
   |        351 x "paragraph" node 
   |      16708 x "sentence" node 
   |      24528 x "subsentence" node 
   |        126 x "text" node 
   |      93763 x "word" node 
   |     833536 nodes of all types
   |       25s OK
   |     0.10s Remo